In [ ]:
import os,sys
import numpy as np
import ROOT
import commentjson
import PlotUtils

In [ ]:

if ("root" in sys.argv[1]): 
    configloc = "root"
    print("getting config from root input " , sys.argv[1] )
    inputname = sys.argv[1]
    f = TFile::Open(inputname, "READONLY")
   

    allconfigs["main"] = (f.Get("main").GetTitle())
    allconfigs["varsFile"] = (f.Get("varsFile").GetTitle())
    allconfigs["cutsFile"] = (f.Get("cutsFile").GetTitle())
    allconfigs["samplesFile"] = (f.Get("samplesFile").GetTitle())
    

    singlesample = 0;
    # see if the root file has already had fits done - these will be used in the cross section fit.
    hasbkgsub = f.Get("fitconfig") != 0;

    if len(sys.argv)  > 1 :
        usetune = argv[2]
    
    else:
    # this is the old way
        print("getting config from command line " , sys.argv[1] )
        configloc = "disk"
        int prescale = 100
        if (argc > 2):
            inputtag += "_" + (sys.argv[2]);
        
    std::string asample;

    if (argc > 3) {
        asample = (argv[3]);
        singlesample = 1;
    } else {
        singlesample = 0;
    }
    NuConfig* theconfig;
    theconfig.Read((argv[1]) + ".json");
    allconfigs["main"] = theconfig;
    inputname = theconfig.GetString("outRoot") + "_" + inputtag + ".root";
    f = TFile::Open(inputname.c_str(), "READONLY");
    # f.ls();
}

    # allconfigs["main"].Print();

    # code used to use config now use allconfigs["main"]
    std::vector<std::string> AnalyzeVariables = allconfigs["main"].GetStringVector("AnalyzeVariables");
    std::vector<std::string> AnalyzeVariables2D = allconfigs["main"].GetStringVector("Analyze2DVariables");
    std::vector<std::string> SampleRequest;

    # either get sample from command line or from the list in the config
    if (!singlesample) {
        SampleRequest = allconfigs["main"].GetStringVector("runsamples");
    } else {
        SampleRequest.push_back(asample);
    }
    std::string playlist = allconfigs["main"].GetString("playlist");

    int m_fluxUniverses = allconfigs["main"].GetInt("fluxUniverses");

    #========================================= Now do some analysis

    MnvH1D* h_flux_dewidthed = GetFlux(allconfigs);

    double flux = h_flux_dewidthed.Integral();
    # make containers for different analysis levels
    std::map<std::string, MnvH1D*> h_flux_ebins;

    # now loop over histograms, unsmear, efficiency correct and normalize

    # unfolding setup
    MinervaUnfold::MnvUnfold unfold;
    # MnvH2D* MigrationMatrix;
    double num_iter = 5;

    # input 4 D hist-map

    # arguments will be sample, variable, type, category
    # order in histogram name is sample___category___variable___type
    # type is reconstructed, truth...
    # category is qelike, qelikenot ...

    std::map<std::string, std::map<std::string, std::map<std::string, std::map<std::string, MnvH1D*> > > > hists1D;

    std::map<std::string, std::map<std::string, std::map<std::string, std::map<std::string, MnvH2D*> > > > hists2D;

    std::map<std::string, std::map<std::string, std::map<std::string, std::map<std::string, MnvH2D*> > > > response1D;

    std::map<std::string, std::map<std::string, std::map<std::string, std::map<std::string, MnvH2D*> > > > response2D;

    # f.ls();

    std::vector<std::string> keys;

    MnvH1D* null = new MnvH1D("null", "empty histogram", 1, 0., 1.);
    MnvH2D* null2 = new MnvH2D("null2", "empty histogram", 1, 0., 1., 1, 0., 1.);

    std::vector<std::string> samples;
    std::vector<std::string> variables;
    std::vector<std::string> categories;
    std::vector<std::string> types;

    # this is the old read in the POT.  They are also in a vector which Amit like having
    TH1D* h_pot = (TH1D*)f.Get("POT_summary");
    h_pot.Print("ALL");

    double dataPOT = h_pot.GetBinContent(1);
    double mcPOTprescaled = h_pot.GetBinContent(3);
    double POTScale = dataPOT / mcPOTprescaled;
    delete h_pot;
    double norm = 1. / dataPOT / targets;
    TNamed targetobj("targets", Form("%6e", targets));

    print(" integrated luminosity is " , 1 / norm / 1.E24 , "barns^-1" )

    print(" POT MC " , mcPOTprescaled )
    print(" POT DATA " , dataPOT )

    for (auto k : *f.GetListOfKeys()) {
        keys.push_back(k.GetName());
        std::string key = k.GetName
        std::string hNd = parsekey[0];
        std::string sample = parsekey[1];
        std::string category = parsekey[2];
        std::string variable = parsekey[3];
        std::string type = parsekey[4];
        
        if (type.find("type")!= string::npos) {
            print(" found a type" , type )
            continue;
        }
        
        
        
         

        # build lists of all valid tags
        # only count sample that you requested
        if (!checktag(SampleRequest, sample)) {
            print(" skip a sample:" , sample )
            continue;
        }
        # build lists of tags
        if (!checktag(samples, sample)) {
            samples.push_back(sample);
            print(" add a sample: " , sample )
        }

        if (!checktag(categories, category)) {
            categories.push_back(category);
            print(" add a category: " , category )
        }
        if (!checktag(variables, variable)) {
            variables.push_back(variable);
            print(" add a variable: " , variable )
        }
        if (!checktag(types, type)) {
            types.push_back(type);
            print(" add a type: " , type )
        }
        # get the histogram
        TNamed* me = f.GetKey(key.c_str());
        print(" Done with parsing" )

        # 1D hists
        if (key.find("h2D") == std::string::npos) {
            # if response in name its a 2D so do a cast to MnvH2D
            if (key.find("migration") != std::string::npos) {
                MnvH2D* hist = (MnvH2D*)(f.Get(key.c_str()));
                if (hist != 0) {
                    response1D[sample][variable][type][category] = hist.Clone();
                    # response1D[sample][variable][type][category].Print();
                    response1D[sample][variable][type][category].SetDirectory(0);
                    print(" migration " , sample , " " , variable , " " , type , " " , category )
                    delete hist;
                } else {
                    print("could not read " , key )
                    response1D[sample][variable][type][category] = 0;
                }
            }
            # it's normal so it's a 1D.
            else {
                MnvH1D* temp = (MnvH1D*)(f.Get(key.c_str()));
                if (temp != 0) {
                    hists1D[sample][variable][type][category] = temp.Clone();
                    hists1D[sample][variable][type][category].Print();
                    hists1D[sample][variable][type][category].SetDirectory(0);
                    print(" hist 1D " , sample , " " , variable , " " , type , " " , category , " " , hists1D[sample][variable][type][category].GetName() )
                    delete temp;
                } else {
                    print("could not read " , key )
                    hists1D[sample][variable][type][category] = 0;
                }
            }
        }

        # 2D hists
        else if (hNd == "h2D" || key.find("h2D") != std::string::npos) {
            std::vector<std::string> partsof2D = split(variable, "_");
            MnvH2D* hist = (MnvH2D*)(f.Get(key.c_str()));
            # Check if response is in its name
            if (hist != 0) {
                if (key.find("migration") != std::string::npos) {
                    response2D[sample][variable][type][category] = hist.Clone();
                    response2D[sample][variable][type][category].Print();
                    response2D[sample][variable][type][category].SetDirectory(0);
                    print(" 2D migration " , sample , " " , variable , " " , type , " " , category )
                    delete hist;
                } else {
                    hists2D[sample][variable][type][category] = hist.Clone();
                    hists2D[sample][variable][type][category].Print();
                    hists2D[sample][variable][type][category].SetDirectory(0);
                    print(" hist 2D " , sample , " " , variable , " " , type , " " , category )
                    delete hist;
                }
            } else {
                print("could not read " , key )
                if (key.find("migration") != std::string::npos) {
                    response2D[sample][variable][type][category] = 0;
                } else {
                    hists2D[sample][variable][type][category] = 0;
                }
            }
        }
    }

    # define the output file
    std::string pdfname;
    std::string outroot;
    std::string stune = "_untuned";
    if (usetune) stune = "_tuned";
    if (singlesample) {
        std::string outname = inputname.replace(inputname.end() - 5, inputname.end(), "") + "_" + asample + stune + "_analyze9";
        outroot = outname + ".root";

        # set up the outputs
        pdfname = outname;
    } else {
        std::string outname = inputname.replace(inputname.end() - 5, inputname.end(), "") + stune + "_analyze9";
        outroot = outname + ".root";

        # set up the outputs
        pdfname = outname;
    }
    TFile* o = TFile::Open(outroot.c_str(), "RECREATE");
    targetobj.Write();
    h_flux_dewidthed.Write();
    std::string pdffilename1D = pdfname + "_1D.pdf";
    std::string pdfstart1D = pdfname + "_1D.pdf(";
    std::string pdfend1D = pdfname + "_1D.pdf)";

    std::string pdffilename1Dres = pdfname + "_1Dres.pdf";
    std::string pdfstart1Dres = pdfname + "_1Dres.pdf(";
    std::string pdfend1Dres = pdfname + "_1Dres.pdf)";

    std::string pdffilename2D = pdfname + "_2D.pdf";
    std::string pdfstart2D = pdfname + "_2D.pdf(";
    std::string pdfend2D = pdfname + "_2D.pdf)";

    o.cd();
    for (auto s : hists1D) {
        auto sample = s.first;
        for (auto v : hists1D[sample]) {
            auto variable = v.first;
            for (auto t : hists1D[sample][variable]) {
                auto type = t.first;
                for (auto c : hists1D[sample][variable][type]) {
                    auto category = c.first;
                    print(" write out " , hists1D[sample][variable][type][category].GetName() )
                    hists1D[sample][variable][type][category].Write();
                }
            }
        }
    }

    o.cd();
    for (auto config : allconfigs) {
        print(" write out config " , config.first )
        std::string obj = config.second.ToString();
        TNamed object(config.first, obj.c_str());
        object.Write();
    }
    # Set up pdf for 1D plots.
    TCanvas canvas1D(pdffilename1D.c_str());
    canvas1D.SetLeftMargin(0.15);
    canvas1D.SetRightMargin(0.15);
    canvas1D.SetBottomMargin(0.15);
    canvas1D.SetTopMargin(0.15);
    canvas1D.Print(pdfstart1D.c_str(), "pdf");

    # bool binwid = true;  # flag you need if MnvPlotter does not do binwid correction.  If it does not, you need to set this to true

    print(" just before 1D loop" )
    for (auto samples : hists1D) {
        std::string sample = samples.first;
        print(" Sample: " , sample )
        for (auto variables : hists1D[sample]) {  # only do this for a subset to save output time.

            std::string variable = variables.first;
            if (!checktag(AnalyzeVariables, variable)) {
                print(" don't do this variable for now" , variable )
                continue;
            }
            print("  Variable: " , variable )
            std::string basename = "h_" + sample + "_" + variable;
            if (singlesample) {
                basename = "h_" + variable;
            }
            print("basename is " , basename )
            int exit = GetCrossSection(sample, variable, basename, hists1D[sample][variable], response1D[sample][variable], allconfigs, canvas1D, norm, POTScale, h_flux_dewidthed, unfold, num_iter, DEBUG, hasbkgsub, usetune, pdfname);

       

In [ ]:
#def gethists(config):
configname = "./Try.json"
f = open(configname,'r')
config = commentjson.load(f)
dataHist = {}
fitHists = {}
unfitHists = {}



    lowBin = 1
    if "LowBin" in config: 
        lowBin = config["LowBin"]
    
    upperLimit=2.0
    if "UpperLimit" in config:
        upperLimit =config["UpperLimit"]
    
    inputFileName=config["InputFile"]
    outputFileName = config["OutputFile"]
    logPlot = config["LogPlot"]
    logMinimum = config["LogMinimum"]
    sidebands = config["Sidebands"]
    categories = config["Categories"]
    
    # use this to exclude or include sidebands in the global fit
    include = config["IncludeInFit"]
    backgrounds = config["Backgrounds"]
    

    includeInFit={}
    for s in sidebands:
        includeInFit[s] = False
        for  i in include:
            if (s == i): includeInFit[s] = True
        
    
    #varName = config["Variable"]
    fitType = config["FitType"]
    h_template = config["Template"]
    f_template = config["FitTemplate"]

    rebin=1
    if "Rebin" in config:
        rebin = config["Rebin"]

    inputFile = ROOT.TFile.Open(inputFileName,"READ")
    outputfile = ROOT.TFile.Open(outputFileName,"RECREATE")
    #loop on all entries of this directory
    
    outputfile.cd()
    
    pot_summary = inputFile.Get("POT_summary")
    potinfo = np.array([0.0,0.0])
    potinfo[0]=pot_summary.GetBinContent(1)
    potinfo[1]=pot_summary.GetBinContent(3) # this includes any prescale
    pot_summary.Print("ALL")
    mcPOT = potinfo[1]
    dataPOT = potinfo[0]
    print ("POT", dataPOT,mcPOT)
    POTscale = potinfo[0]/potinfo[1]
  
    print ("POT scale factor: " , POTscale)
    
    # make and fill maps that contain poers to the histograms you want to fit  uses CCQEMAT template
    
    #h_template = "h___%s___%s___%s___reconstructed";
    # dataHist = {}
    # fitHists = {}
    # unfitHists = {}
    
    name = config["Variable"]

    for side in sidebands:
        cat = "data"
        cname = h_template%(side, cat,varName)
        cname = h_template%(side, cat,varName)
        fname = f_template%(side, cat,varName)
        print ( " look for " , cname )
        dataHist[side] = PlotUtils.MnvH1D()
        dataHist[side] = inputFile.Get(cname)
        
        dataHist[side].Rebin(rebin)
        dataHist[side].Write()
        unfitHists[side] = []
        fitHists[side] = []
        print ( " nbins " , cname , " " , dataHist[side].GetXaxis().GetNbins() )
        if (logPlot): dataHist[side].SetMinimum(logMinimum)
        #dataHist[side].SetNormBinWidth(1.0);
        #dataHist[sidename] = (TH1D*)dataHist.GetCVHistoWithStatError().Clone();
        for  cat in categories:
            cname = h_template%(side, cat,varName)
            fname = f_template%(side, cat,varName)
            name = cname
            print ( " look for " , cname )
            newhist = PlotUtils.MnvH1D()
            newhist = inputFile.Get(cname)
            if not newhist:
                print ( " no " , cname )
            
            newhist.Rebin(rebin);
            if (logPlot): newhist.SetMinimum(logMinimum)
            print ( "nbins " , cname , " " , newhist.GetXaxis().GetNbins() )
            
            newhist.Scale(POTscale)
            newhist.Print()
            newhist.Write()
            unfitHists[side].append(newhist)
            fitHists[side].append(newhist.Clone(fname))
            
        
        # /*for ( i = 0; i < categories.size(); i++){
        #     unfitHists[side][i].SetNormBinWidth(1.0);
        #     fitHists[side][i].SetNormBinWidth(1.0);
        # }*/

return unFithists,fitHists    
  

In [ ]:
def rescale()

In [ ]:
configname = "./Try.json"
f = open(configname,'r')
config = commentjson.load(f)
gethists(config)


In [ ]:
values = np.array([1.28121225775791303e+00,9.26752433073161686e-01,2.34814205775936458e+00,1.19256001977170878e+00,2.06809555409070267e+00])

In [ ]:
testarray = [[4.37913794018177847e-05,-2.70534311853663019e-04,7.68293601191285961e-04,-1.01035963754613244e-06,-6.33620216728098760e-05],
[-2.70534311853663019e-04,3.72604389054405870e-03,-1.18219944391336363e-02,-4.50352671609323995e-05,7.55635486379529001e-03],
[7.68293601191285961e-04,-1.18219944391336363e-02,4.08349797094741951e-02,1.10370908939183272e-03,-7.19945013816742918e-02],
[-1.01035963754613244e-06,-4.50352671609323995e-05,1.10370908939183272e-03,7.35760891367070124e-04,-2.95373032784254745e-02],
[-6.33620216728098760e-05,7.55635486379529001e-03,-7.19945013816742918e-02,-2.95373032784254745e-02,1.25567293391937773e+00]]

testarray = [[4.51788475382673715e-05,-2.85364565733574869e-04,8.06084163862019428e-04,1.97705040158106509e-06,-8.40679740034714436e-05],
[-2.85364565733574869e-04,3.93866790473751654e-03,-1.25468028770457200e-02,-1.61393981126265563e-04,1.16813750019919144e-02],
[8.06084163862019428e-04,-1.25468028770457200e-02,4.43479016837736995e-02,2.13451152907324221e-03,-1.14147245052297855e-01],
[1.97705040158106509e-06,-1.61393981126265563e-04,2.13451152907324221e-03,1.29665819930366560e-03,-5.22224042559752202e-02],
[-8.40679740034714436e-05,1.16813750019919144e-02,-1.14147245052297855e-01,-5.22224042559752202e-02,2.19307068295353469e+00]]

In [ ]:
print (testarray)

In [ ]:
testvalues = [1.27933918566519456e+00,9.71067727320098717e-01,2.22202915442571713e+00,1.13399764790330382e+00,3.68119190607679014e+00]

In [ ]:
errors = np.zeros(5)
for i in range(0,5):
    errors[i] = np.sqrt(testarray[i][i])

pr (errors)

array = np.array(testarray)
pr (array)

correlations = array.copy()
for i in range(0,5):
    for j in range(0,5):
        correlations[i][j] = array[i][j]/errors[i]/errors[j]

pr (correlations)



parameters = np.array(testvalues)

In [ ]:
eigvalues,eigvectors = np.linalg.eig(array)
pr (eigvalues)
pr (eigvectors)
pr (np.round(eigvectors*100))

In [ ]:
variants = []
pr (values)
 
for x in range(0,len(values)):
    #pr ("vec",eigvectors[x])
    vec = eigvectors[x]
    new = eigvectors[x]*np.sqrt(eigvalues[x])
    variants.append(new)
    pr ("new",new)
    


In [ ]:
Print(testvalues)
Print(testvalues + variants[0])
Print(testvalues - variants[0])